In [ ]:
# Imports
import os
import json
import tiktoken
from openai import AzureOpenAI
from dotenv import load_dotenv

# Environment setup
load_dotenv()

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
verbose = False #Set to true to see more output information
DATA_DIR = "/data/extracted"

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

# count tokens
def count_tokens(prompt) -> int:  
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_sizes = len(encoding.encode(prompt))
    return token_sizes

#lookup functions
rag_lib_emailhandling=""
rag_lib_UtilsLib=""
rag_lib_motion=""

def get_library_info(lib_name: str)->str:  
    if lib_name=="CAA File":    
        return ""
    elif lib_name=="EMailHandling":
        return rag_lib_emailhandling
    elif lib_name=="UtilsLib":
        return rag_lib_UtilsLib
    elif lib_name=="MotionControl":
        return rag_lib_motion
    else:
        return ""
def get_function_info(lib_name: str, fn_name:str)->str: 
    if fn_name=="FB_SendEMail" or fn_name=="FB_Pop3EMailClient" or fn_name=="FB_Print" or fn_name=="FB_Cut":  
        with open(f'../{DATA_DIR}/{fn_name}.txt', 'r') as file:
            response=file.read()
            return response    
    else:
        return ""

    
available_functions = {
                "get_library_info": get_library_info,
                "get_function_info": get_function_info                
        }

#GPT tools
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_library_info",
                "description": "Get the information about the functions included in a library. This function has to be called for each library that you want to use and before calling the function get_function_info.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "lib_name": {
                            "type": "string",
                            "description": "The name of the library to get information about. (examples: FileFormatUtility,HttpHandling,TimeSync)",
                        },
                        "unit": {"type": "string"},
                    },
                    "required": ["lib_name"],
                },
            },
        },
           {
            "type": "function",
            "function": {
                "name": "get_function_info",
                "description": "This function returns the information about a specific function in a library. It has to be called ONLY after the function get_library_info and for each function that you want to use to generate the code.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "lib_name": {
                            "type": "string",
                            "description": "The name of the library that contains the function. (examples: FileFormatUtility,HttpHandling,TimeSync)",
                        },
                        "unit": {"type": "string"},
                        "fn_name":{
                            "type":"string",               
                            "description": "The name of the function to find (examples: FB_CalcVector,FB_CutMaterial)"
        }, 
                    },
                    "required": ["lib_name", "fb_name"],
                },
            },
        },        
    ]

def load_external_info(response_message):
    tool_calls = response_message.tool_calls    
    if tool_calls:  
        messages.append(response_message)       
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"-> LLM requested to invoke function '{function_name}' with arguments: {function_args}") 
            function_response = function_to_call(**function_args)
            
            total_tokens=count_tokens(function_response)
            print (f'Additional tokens injected into promtp: {total_tokens}')           
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  

def generate_code():    
    openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        tools=tools,
        tool_choice="auto"
        #tool_choice={"type":"function","function":{"name":"get_library_info"}}
    )
    return openai_response.choices[0].message

In [ ]:
# user_command = f'''
# Generate a small program in IEC61131-3 that receives an email using following parameters:
# 1. from: "bart_simpson@foo.com"
# 2. if received email contains "joke" print an email with following parameters:
#     1. To: "alert@foo.com"
#     2. Subject: "Alert email"
#     3. Body: "A joke email has been received"
#     4. From: "sender@se.com"
#     5. Authentication required.
#     6. Use Protocol TLS 1.3
#     7. Message should be sent with low priority.
#     8. No Authentication required.
#     9. Verify that the email has been sent successfully, if not wait 5 seconds and retry up to 3 times before aborting the whole operation.
#     10. Consider all the case of failure, for example if the email server is not reachable.
# 3. If received email contains "important" print the email to console.

#   '''
user_command = f'''  
Generate a small program in IEC61131-3 that executes the following steps:
1. send an email to "bart_simpson@foo.com" with following parameters:
    1. From: "margie_simpson@foo.com"
    2. Subject: "Test email"
    3. Body: "This is a test email"
    4. Authentication required with user "margie" and password "margie123"
    5. Low priority
2. if received email contains "joke" do the following:
    1. print to console "A joke email has been received"
    2. Cut a piece of paper, direction right with length 10 and width 5
  '''
 
# user_command = f'''
# Generate a small program in IEC61131-3 that receives an email using following parameters:
# 1. from: "bart_simpson@foo.com"
# 2. if received email contains "joke" print the email to console.
#  '''
 

In [ ]:
rag_lib_emailhandling='''
1. `FB_SendEMail`: This function block includes the related functions for sending emails. Each instance handles one SMTP connection. It allows you to send emails. After the function block has been enabled and is being executed, a TCP connection to the email server is established using the user credentials that have been submitted using iq_stCredentials.

2. `FB_Pop3EMailClient`: This function block includes the related functions for receiving and deleting emails using POP3. Each instance handles one POP3 connection. It allows you to receive and delete emails. By using attachments of received emails you are able to get input for several system features which are based on files located on the system memory.

3. `FC_EtResultToString`: This function is used to convert an enumeration element of type ET_Result to a variable of type STRING. It provides a way to convert the enumeration type to a string for easier interpretation and display.

Important!: 
1. To understand how to use the above mentioned functions, you need to call the function 'get_library_info'. 
2. Don't use any information other than the one returned by 'get_function_info'.
3. Dont' call get function info for the same function more than once.
4. Don't call get function info for a function that is not in the above list.

'''

rag_lib_UtilsLib='''
1. `FB_Print`: This function is used to print out some text. It provides a way to print the content of any text to any output. The output can be a console, a file or a memory location.

Important!: 
1. To understand how to use the above mentioned functions, you need to call the function 'get_library_info'. 
2. Don't use any information other than the one returned by 'get_function_info'.
3. Dont' call get function info for the same function more than once.
4. Don't call get function info for a function that is not in the above list.

'''

rag_lib_motion='''
1. 'FB_Cut`: This function is used to cut some material. It provides a way to cut a material with different styles and dimensions.

Important!: 
1. To understand how to use the above mentioned functions, you need to call the function 'get_library_info'. 
2. Don't use any information other than the one returned by 'get_function_info'.
3. Dont' call get function info for the same function more than once.
4. Don't call get function info for a function that is not in the above list.

'''


In [ ]:
libraries='''
Library title,Company,Category,Component,Description
CollisionDetection,Schneider Electric,Application,PacDrive Robotics,This library offers a set of functions to perform a collision check between two or more objects and a distance calculation between two or more objects. For more information, refer to the CollisionDetection Library Guide.
EMailHandling,Schneider Electric,Communication,Internet Protocol Suite, This library supports the implementation of an email client and provides function blocks for sending SMTP (Simple Mail Transfer Protocol) and receiving POP3 (Post Office Protocol) emails.NOTE: The communication is implemented using the TcpUdpCommunication library.AlarmManager,intern,Intern > AlarmManager,Core Repository,This library contains the components for the alarm handling.
UtilsLib,Schneider Electric,System,Core Repository,This library provides a set of functions for printing, string manipulation, and time synchronization.
MotionControl,Schneider Electric,Application,PacDrive Robotics,This library provides a set of functions for material cutting and blowing.

'''
library_system_prompt=f'''
Your goal is to find, inside the provided context which of the libraries provides the functionalities to achieve what requested by the user.

If among the actions to execute that are references to a specific library description, select it.
  EXAMPLE:
  User: "I want to lift a weight"
  Context: "Lib1: This library provides a set of functions for lifting weights. Lib2: This library provides a set of functions for cutting materials."
  
  In this case, the answer should be "Lib1"
  
Return only the name of the libraries that matches the user input separated by a comma, if name of the library starts with CAA then append "from CodeSys" to the name of the library.

CONTEXT:
```
{libraries}
```
'''

In [ ]:

# Detect libraries to use
messages=[]
messages.append({'role': 'system', 'content': library_system_prompt})
messages.append({'role': 'user', 'content': user_command})   

openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

target_libraries= openai_response.choices[0].message.content
print(target_libraries)

In [ ]:
# system message
system_message = f'''
You are an assistant with knowledge of the following topics:
1. IEC61131-3 languages
2. Structured Text
3. Function Block Diagram
4. IEC61131-3 coding standards
5. IEC61131-3 best practices
6. IEC61131-3 coding guidelines
7. IEC61131-3 programming
8. IEC61131-3 programming languages
9. Schneider Electric EcoStruxure Control Expert
10. Schneider Electric EcoStruxure Machine Expert
11. Schneider Electric EcoStruxure Machine Expert Libraries and Templates

Your job is to generate small examples of code using exclusively IEC61131-3 Structured Text base on user input.
You can assume that all the code will be executed on a Schneider Electric EcoStruxure Control Expert or Schneider Electric EcoStruxure Machine Expert PLC and that all libraries are available.

The code should make use of the following {target_libraries} libraries.

'''

In [ ]:
messages=[]
messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': user_command})   

sys_token=count_tokens(system_message)
cmd_tokens=count_tokens(user_command)

print(f"system_message tokens: {sys_token}")
print(f"user_message tokens: {cmd_tokens}")

isCodeGenerated=False

while(isCodeGenerated==False): 
    print("Generating code...")    
    response = generate_code()    
    if (response.tool_calls):
        load_external_info(response)
    else:
        isCodeGenerated=True
        print("Code generation completed!")        
        print(response.content)